# Piping a Prompt, Model, and an Output Parser

In [1]:
pip show langchain

Name: langchain
Version: 1.2.7
Summary: Building applications with LLMs through composability
Home-page: https://docs.langchain.com/
Author: 
Author-email: 
License: MIT
Location: c:\Users\Theo\source\repos\AUEB\LangChain\.venv\Lib\site-packages
Requires: langchain-core, langgraph, pydantic
Required-by: 
Note: you may need to restart the kernel to use updated packages.


In [2]:
from langchain_openai import ChatOpenAI

from langchain_core.prompts import ChatPromptTemplate
from langchain_core.output_parsers import CommaSeparatedListOutputParser

c:\Users\Theo\source\repos\AUEB\LangChain\.venv\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [3]:
list_instructions = CommaSeparatedListOutputParser().get_format_instructions()

In [4]:
list_instructions

'Your response should be a list of comma separated values, eg: `foo, bar, baz` or `foo,bar,baz`'

In [5]:
chat_template = ChatPromptTemplate.from_messages([
    ('human', 
     "I've recently adopted a {pet}. Could you suggest three {pet} names? \n" + list_instructions)])

In [6]:
print(chat_template.messages[0].prompt.template)

I've recently adopted a {pet}. Could you suggest three {pet} names? 
Your response should be a list of comma separated values, eg: `foo, bar, baz` or `foo,bar,baz`


In [7]:
chat = ChatOpenAI(model_name = 'gpt-4', 
                  model_kwargs = {'seed':365},
                  temperature = 0,
                  max_tokens = 100)

c:\Users\Theo\source\repos\AUEB\LangChain\.venv\Lib\site-packages\IPython\core\interactiveshell.py:3641: UserWarning: Parameters {'seed'} should be specified explicitly. Instead they were passed in as part of `model_kwargs` parameter.
  if await self.run_code(code, result, async_=asy):


In [8]:
list_output_parser = CommaSeparatedListOutputParser()

In [10]:
chat_template_result = chat_template.invoke({'pet':'dog'})
chat_template_result

ChatPromptValue(messages=[HumanMessage(content="I've recently adopted a dog. Could you suggest three dog names? \nYour response should be a list of comma separated values, eg: `foo, bar, baz` or `foo,bar,baz`", additional_kwargs={}, response_metadata={})])

In [12]:
chat_result = chat.invoke(chat_template_result)
chat_result

AIMessage(content='Max, Bella, Charlie', additional_kwargs={'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 5, 'prompt_tokens': 50, 'total_tokens': 55, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens': 0, 'rejected_prediction_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}}, 'model_provider': 'openai', 'model_name': 'gpt-4-0613', 'system_fingerprint': None, 'id': 'chatcmpl-D2euYUQw2mqtY98Q2czBPQZXcUIMJ', 'service_tier': 'default', 'finish_reason': 'stop', 'logprobs': None}, id='lc_run--019bfffc-61c0-7c52-bb01-e41fc7bea93d-0', tool_calls=[], invalid_tool_calls=[], usage_metadata={'input_tokens': 50, 'output_tokens': 5, 'total_tokens': 55, 'input_token_details': {'audio': 0, 'cache_read': 0}, 'output_token_details': {'audio': 0, 'reasoning': 0}})

In [13]:
list_output_parser.invoke(chat_result)

['Max', 'Bella', 'Charlie']

In [14]:
chain = chat_template | chat | list_output_parser

In [15]:
chain.invoke({'pet':'dog'})

['Max', 'Bella', 'Charlie']